<a href="https://colab.research.google.com/github/jinkyukim-me/Sentiment-analysis-pytorch/blob/master/Upgraded_Sentiment_jin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개선된 감성분석(Upgrade Sentiment)
### 단순 감성분석(Simple Sentiment.ipynb)에 더해질 것들

* packed padded sequences
* pre-trained word embeddings
* different RNN architecture
* bidirectional RNN
* multi-layer RNN
* regularization
* a different optimizer



## 데이터 준비

* import module
* sets the seed for generating random numbers
* define the Fields 
* get the train/valid/test splits.

`packed padded sequences`을 적용<br>
`TEXT` field에 include_lengths = True 적용


In [0]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

* TEXT field to define how the review should be processed, and the LABEL field to process the sentiment

In [0]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

* load the IMDb dataset.
* splits it into the canonical train/test splits as torchtext.datasets objects.<br>
* The IMDb dataset consists of 50,000 movie reviews, each marked as being a positive or negative review.

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.7MB/s]


* create the validation set from our training set.

In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

### the use of pre-trained word embeddings. 

* using the `"glove.6B.100d" vectors"`. glove is the algorithm used to calculate the vectors, go here for more. `6B` indicates these vectors were trained on 6 billion tokens and `100d` indicates these vectors are 100-dimensional.

* Add
    ```
    vectors = "glove.6B.100d", 
    unk_init = torch.Tensor.normal_
    ```


In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [00:43, 19.9MB/s]                           
100%|█████████▉| 398174/400000 [00:20<00:00, 19278.47it/s]

### placing the tensors on the GPU if one is available.

* Another thing for packed padded sequences all of the tensors within a batch need to be sorted by their lengths. 

* Add
    ```
    sort_within_batch = True
    ```

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

## Build the Model

* Building the model that we'll eventually train and evaluate.
* Add
    ```
    n_layers, bidirectional, dropout, pad_idx
    
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
    
    packed_output, (hidden, cell) = self.rnn(packed_embedded)
    
    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
    ```
    


In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

* Add parameters

    ```
    N_LAYERS = 2
    BIDIRECTIONAL = True
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
    ```

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
            N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

* Create a function that will tell us how many trainable parameters our model has so we can compare the number of parameters across different models.

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [10]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [11]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-3.6247, -1.2173, -0.7283,  ..., -0.6435, -0.2026, -0.3940],
        [-0.6058, -0.2772,  1.4801,  ..., -0.1984,  0.5430,  1.5206],
        [-0.5121,  0.2059,  0.5721,  ..., -0.7374, -0.2360,  0.3210]])

In [12]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-3.6247, -1.2173, -0.7283,  ..., -0.6435, -0.2026, -0.3940],
        [-0.6058, -0.2772,  1.4801,  ..., -0.1984,  0.5430,  1.5206],
        [-0.5121,  0.2059,  0.5721,  ..., -0.7374, -0.2360,  0.3210]])


##Train the Model
* Set up the training and then train the model

* Create an optimizer
* Use stochastic gradient descent (SGD)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

* Define our loss function. 
* In PyTorch this is commonly called a criterion.

* The loss function here is binary cross entropy with logits.

* The BCEWithLogitsLoss criterion carries out both the sigmoid and the binary cross entropy steps.

In [0]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

* Criterion function calculates the loss 

* This rounds any value greater than 0.5 to 1 (a positive sentiment) and the rest to 0 (a negative sentiment).


In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

* The `train` function iterates over all examples, one batch at a time.


In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

* `evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

* Create a function to tell us how long an epoch takes to compare training times between models.

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

* Train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

In [19]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|█████████▉| 398174/400000 [00:40<00:00, 19278.47it/s]

Epoch: 01 | Epoch Time: 1m 37s
	Train Loss: 0.679 | Train Acc: 57.65%
	 Val. Loss: 0.774 |  Val. Acc: 51.72%
Epoch: 02 | Epoch Time: 1m 37s
	Train Loss: 0.631 | Train Acc: 63.48%
	 Val. Loss: 0.552 |  Val. Acc: 72.25%
Epoch: 03 | Epoch Time: 1m 37s
	Train Loss: 0.483 | Train Acc: 77.29%
	 Val. Loss: 0.447 |  Val. Acc: 81.09%
Epoch: 04 | Epoch Time: 1m 40s
	Train Loss: 0.387 | Train Acc: 83.34%
	 Val. Loss: 0.335 |  Val. Acc: 84.96%
Epoch: 05 | Epoch Time: 1m 40s
	Train Loss: 0.320 | Train Acc: 86.95%
	 Val. Loss: 0.378 |  Val. Acc: 82.80%


* Get new and vastly improved test accuracy!
* Achieve 85.% test accuracy.

In [20]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.334 | Test Acc: 85.39%


* Negative sentiment to return a value close to 0 

* Positive sentiment to return a value close to 1

In [0]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

* An example negative review...

In [29]:
predict_sentiment(model, "This film is worse")

0.43286192417144775

In [30]:
predict_sentiment(model, "This film is terrible")

0.41769811511039734

* An example positive review...

In [28]:
predict_sentiment(model, "This film is really awesome")

0.959578812122345

In [31]:
predict_sentiment(model, "This film is great")

0.94480299949646